In [60]:
import pandas as pd
import os

In [ ]:
path = os.getcwd()
path_trump = path + "\\data\\hashtag_donaldtrump.csv"
trump = pd.read_csv(path_trump, lineterminator="\n")
path_biden = path + "\\data\\hashtag_joebiden.csv"
biden = pd.read_csv(path_biden, lineterminator="\n")
trump["source"] = "Trump"
biden["source"] = "Biden"
# Concatenate and remove duplicates
df = pd.concat([trump, biden], ignore_index=True)
df = df.drop_duplicates()
df = df[df["country"].isin(["United States of America", "United States"])].dropna(
    subset=["state"]
)

In [83]:
frequent_names_trump = [
    "Trump",
    "Donald" "Donald Trump",
    "@realDonaldTrump",
    "The Donald",
    "45",
    "Donald J. Trump",
    "DJT",
    "The Trump Administration",
    "Trumpster",
    "POTUS",
    "@POTUS",
]
frequent_names_biden = [
    "Biden",
    "Joe Biden",
    "@JoeBiden",
    "The Biden",
    "46",
    "Joseph R. Biden",
    "JRB",
    "The Biden Administration",
    "Bidenster",
    "Joe",
    "Joseph",
    "Joseph Biden",
    "Sleepy Joe",
    "Uncle Joe",
]

# Create regex patterns from the lists of names
pattern_trump = "|".join(frequent_names_trump)
pattern_biden = "|".join(frequent_names_biden)

# Create boolean masks where tweets contain any of the frequent names
mask_trump = df["tweet"].str.contains(pattern_trump, case=False, na=False)
mask_biden = df["tweet"].str.contains(pattern_biden, case=False, na=False)

# Combine the masks to filter for tweets containing Trump names but not Biden names or vice versa
filtered_tweets = df[(mask_trump & ~mask_biden) | (mask_biden & ~mask_trump)]

# Count the number of filtered tweets
count_filtered = filtered_tweets.shape[0]

# Total number of tweets for percentage calculation
total_tweets = len(df["tweet"])

# Print the results
print(
    f"Number of tweets containing Trump names but not Biden names, or vice versa: {count_filtered}"
)
print(f"Percentage of filtered tweets: {count_filtered / total_tweets * 100:.2f}%")


filtered_tweets_trump = df[mask_trump & ~mask_biden]
filtered_tweets_biden = df[mask_biden & ~mask_trump]
count_filtered_trump = filtered_tweets_trump.shape[0]
count_filtered_biden = filtered_tweets_biden.shape[0]
print(
    f"\nNumber of tweets containing Trump names but not Biden names: {count_filtered_trump}"
)
print(
    f"Percentage of filtered tweets Trump: {count_filtered_trump / total_tweets * 100:.2f}%"
)
print(
    f"\nNumber of tweets containing Biden names but not Trump names: {count_filtered_biden}"
)
print(
    f"Percentage of filtered tweets Biden: {count_filtered_biden / total_tweets * 100:.2f}%"
)

Number of tweets containing Trump names but not Biden names, or vice versa: 207552
Percentage of filtered tweets: 62.43%

Number of tweets containing Trump names but not Biden names: 119153
Percentage of filtered tweets Trump: 35.84%

Number of tweets containing Biden names but not Trump names: 88399
Percentage of filtered tweets Biden: 26.59%
